In [7]:
import cv2, torch, torchvision
from torch.utils.data import DataLoader

In [8]:
import model
import torch.nn as nn
from torch.optim import Adam
from dataset import FreiHand


In [9]:
COLORMAP = {
    "thumb": {"ids": [0, 1, 2, 3, 4], "color": (0,255,0) #green
             },
    "index": {"ids": [0, 5, 6, 7, 8], "color": (0,255,255) #cyan
             },
    "middle": {"ids": [0, 9, 10, 11, 12], "color": (0,0,255) #blue
              },
    "ring": {"ids": [0, 13, 14, 15, 16], "color": (255,0,255) #pink
            },
    "little": {"ids": [0, 17, 18, 19, 20], "color": (255,0,0) #red
              },
}

In [10]:
config = {
    "data_dir": "Dataset/FreiHand",
    "n_sample": 1,
    "n_joints": 21,
    "epochs": 1,
    "batch_size": 1,
    "shuffle":True,
    "batches_per_epoch": 50,
    "batches_per_epoch_val": 20,
    "learning_rate": 0.001,
    "decay": 0.0001,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}

In [11]:
module = model.Resnet(3,model.ResBottleneckBlock,[3,4,6,3], useBottleneck=True)
module=module.to(config["device"])

In [40]:
%cd T:\CS406\raw model

T:\CS406\raw model


In [12]:
module.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>

In [41]:
# Define the loss function with Classification Cross-Entropy loss and an optimizer with Adam optimizer
loss_fn = nn.MSELoss()
optimizer = Adam(module.parameters(), lr=0.001, weight_decay=0.0001)

In [218]:
train_set = FreiHand(config=config, set_type="train")
train_dl = DataLoader(train_set, batch_size=config["batch_size"], shuffle= True)

val_set = FreiHand(config=config, set_type="val")
val_dl = DataLoader(val_set, batch_size=config["batch_size"], shuffle= False)


In [223]:
for e in range(config["epochs"]):
    loss_epoch=[]
    for _, data in enumerate(train_dl):
        optimizer.zero_grad()
        inputs= data["image"].to(config["device"])
        target= data["heatmaps"].to(config["device"])
        yhat=module(inputs).to(torch.float64)
        yhat = yhat.reshape((config["batch_size"], config["n_joints"], -1)).split(1,1)
        target = target.reshape((config["batch_size"], config["n_joints"], -1)).split(1,1)
        
        final_loss=torch.Tensor([0]).cuda()
        loss=torch.Tensor([0]).cuda()

        for joint in range(config["n_joints"]):
            yhat_joint=yhat[joint].squeeze()
            target_joint=target[joint].squeeze()
            loss += loss_fn(yhat_joint,target_joint)
        print(loss)
        final_loss = 100 * loss
        loss_epoch.append(loss.item())
        #print(final_loss)
        final_loss.backward()
        optimizer.step()

    print('Loss at {}: {}'.format(e,np.mean(loss_epoch)))
    torch.save(module.state_dict(), 'model_weights.pth')

tensor([5.5429], device='cuda:0', grad_fn=<AddBackward0>)
tensor([4.1057], device='cuda:0', grad_fn=<AddBackward0>)
tensor([2.3320], device='cuda:0', grad_fn=<AddBackward0>)
tensor([1.2943], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.7344], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.4496], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.3080], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.2339], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1957], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1733], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1608], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1529], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1486], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1455], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1439], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1429], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1424], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1416

tensor([0.1230], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1278], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1272], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1298], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1287], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1299], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1189], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1220], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1269], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1237], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1280], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1239], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1261], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1248], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1186], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1249], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1207], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1185

tensor([0.1277], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1143], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1261], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1092], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1173], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1139], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1244], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1311], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1305], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1229], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1164], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1270], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1221], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1198], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1166], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1248], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1107], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1124

tensor([0.1135], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1307], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1326], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1190], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1100], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1178], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1076], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1218], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1116], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1160], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1131], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1211], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1170], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1005], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1249], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1319], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1086], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1249

tensor([0.1074], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1059], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1011], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1034], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1065], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0941], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1376], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1249], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1170], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1118], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1075], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1315], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1212], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1124], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1169], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1096], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1291], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1189

...

tensor([0.1119], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0845], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1300], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1143], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0873], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0756], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1188], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0822], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0845], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0995], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1100], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1210], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0843], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1138], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1315], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1099], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0974], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1041

tensor([0.0848], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0926], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1126], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1160], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1378], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0825], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1177], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1114], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0961], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0902], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1150], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0797], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1268], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0986], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1099], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0891], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0829], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1381

tensor([0.1236], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1158], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0998], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1018], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0994], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0848], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0682], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0822], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1118], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1465], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0608], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0806], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0896], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0696], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0623], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0673], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0807], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1221

tensor([0.0824], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0986], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1145], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0935], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0859], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0763], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0998], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0720], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0822], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1508], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0880], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1010], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0828], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1347], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0867], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0951], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0989], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0973

tensor([0.0682], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1120], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0707], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0566], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0923], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0869], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1104], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0880], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1276], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1114], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1327], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1079], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0865], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0794], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0568], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1118], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1183], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1057

In [53]:
import matplotlib.pyplot as plt
import numpy as np

In [202]:
target=train_set[0]['heatmaps'].unsqueeze(0).reshape(1,21,-1).split(1,1)

In [192]:
yhat=module(train_set[0]["image"].unsqueeze(0).to('cuda'))

In [194]:
yhat=yhat.reshape(1,21,-1).split(1,1)

In [212]:
loss=torch.Tensor([0]).to(torch.double).cuda()
for i in range(21):
    l=loss_fn(yhat[i].squeeze().cuda(), target[i].squeeze().cuda())
    print(l)
    loss+=l

tensor(0.3174, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.3081, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2272, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2146, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2914, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2136, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2473, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.3541, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2470, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2462, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2927, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2493, device='cuda:0', 

RuntimeError: Found dtype Double but expected Float

In [156]:
ran=torch.randn((21,64,64)).reshape(21,-1).split(1,0)[0].squeeze()

In [ ]:
for _ in range(config["epochs"]):
    for i, data in enumerate(train_dl):
        optimizer.zero_grad()
        inputs= data["image"].to(config["device"])
        targets= data["heatmaps"].to(config["device"])
        yhat=module(inputs).to(torch.float64)

        loss= loss_fn(yhat, targets)
        print(loss)
        loss.backward()
        optimizer.step()

tensor(0.2368, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2193, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2014, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.1853, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.1740, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.1418, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.1250, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.1094, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0922, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0794, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0699, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0624, device='cuda:0', 

tensor(0.0230, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0136, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0117, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0139, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0142, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0124, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0140, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0271, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0137, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0190, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0153, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0177, device='cuda:0', 

In [ ]:
bla=train_set[0]['keypoints'].detach().cpu().numpy()

In [ ]:
bla

In [ ]:
bla=bla.reshape(2,21)*32

In [ ]:
from skimage.transform import resize
bla2 = resize(train_set[0]['image'].detach().cpu().numpy().transpose([2, 1, 0]), (32, 32))

In [ ]:
bla2

In [ ]:
import matplotlib.pyplot as plt


plt.subplot(1,2,2)
plt.imshow(train_set[0]['heatmaps'][17])
plt.subplot(1,2,1)
plt.imshow(bla2)
plt.scatter(bla[0][17],bla[1][17])

In [ ]:
output

In [3]:
from PIL import Image
from torchvision import transforms
bla=Image.open("test.jpg")

C:\Users\hoang\anaconda3\envs\ComputerVision\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
image_transform = transforms.Compose([
				transforms.Resize(128),
				transforms.ToTensor(),
				])

In [5]:
bla1= image_transform(bla)

In [14]:
def heatmaps_to_coordinates(heatmaps):
    """
    Heatmaps is a numpy array
    Its size - (batch_size, n_keypoints, img_size, img_size)
    """
    batch_size = heatmaps.shape[0]
    sums = heatmaps.sum(axis=-1).sum(axis=-1)
    sums = np.expand_dims(sums, [2, 3])
    normalized = heatmaps / sums
    x_prob = normalized.sum(axis=2)
    y_prob = normalized.sum(axis=3)

    arr = np.tile(np.float32(np.arange(0, 64)), [batch_size, 21, 1])
    x = (arr * x_prob).sum(axis=2)
    y = (arr * y_prob).sum(axis=2)
    keypoints = np.stack([x, y], axis=-1)
    return keypoints / 64

In [15]:
cap= cv2.VideoCapture(0)
while (cap.isOpened()):
    ret, frame= cap.read()
    if ret==True:
        frame = cv2.flip(frame,1)
        input = frame.copy()
        input = cv2.resize(input, (128, 128))
        input = torch.tensor(input.transpose([2,0,1]), dtype= torch.float, device = 'cuda') #hwd -> dhw
        #mean, std = input.mean([1,2]), input.std([1,2])
        mean, std= [0.5, 0.5, 0.5], [1, 1, 1]
        input = torchvision.transforms.functional.normalize(input, mean, std)
        result = module(input.unsqueeze(0))
        hmap=result.cpu().detach().numpy()
        keypoints= heatmaps_to_coordinates(hmap)
        keypoints=keypoints.reshape(21,2)
        bla=np.c_[keypoints[:,1]*640,keypoints[:,0]*480]
        for finger, params in COLORMAP.items():
            #cv2.circle()
            cv2.polylines(frame,[np.int32(np.c_[bla[params["ids"],0],bla[params["ids"],1]])],0,params["color"])
        cv2.imshow('result',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
             break
    else:
        break